In [7]:
from transformers import pipeline
model_name= "Qwen/Qwen2.5-3B-Instruct"

In [8]:

base_chat = pipeline(model = model_name)

config.json:   0%|          | 0.00/661 [00:00<?, ?B/s]

model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/2.20G [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/3.97G [00:00<?, ?B/s]

...

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

Device set to use cuda:0


In [9]:

print(base_chat("Are you familiar with mariya sha??")[0]['generated_text'])

Are you familiar with mariya sha?? I'm not familiar with "Mariya Sha" as a specific person or entity. It's possible you might be referring to:

1. Mariya Shapovalova - A Ukrainian politician who served as a member of the Verkhovna Rada (parliament) from 2006 to 2012.
2. Mariya Shapoval - Another Ukrainian politician who has been involved in politics and advocacy work.
3. There could be other individuals named Mariya Sha, but without more context, it's difficult to determine which one, if any, you're referring to.

If you have more information about Mariya Sha, such as their profession, location, or a specific context where they appear, I'd be happy to try to provide more accurate information. Otherwise, it might be helpful to rephrase your question or provide additional details so I can better understand who you're asking about.


In [11]:
from datasets import load_dataset
raw_data = load_dataset('json', data_files='/content/mariya.json')
raw_data

Generating train split: 0 examples [00:00, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['prompt', 'completion'],
        num_rows: 236
    })
})

In [12]:
raw_data.column_names

{'train': ['prompt', 'completion']}

In [13]:
raw_data['train']

Dataset({
    features: ['prompt', 'completion'],
    num_rows: 236
})

In [14]:
raw_data['train'][0]

{'prompt': 'Who is  Mariya Sha ?',
 'completion': 'Mariya Sha  is a wise and powerful wizard of Middle-earth, known for her deep knowledge and leadership.'}

In [15]:
# Tokenization

In [16]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [17]:
def preprocessing_raw_data(sample):
  sample = sample['prompt'] + '\n' + sample['completion']
  tokenized = tokenizer(sample, max_length = 128, truncation = True,
                        padding= 'max_length')
  tokenized['labels']= tokenized['input_ids'].copy()
  return tokenized



In [18]:
dataset_ = raw_data.map(preprocessing_raw_data)

Map:   0%|          | 0/236 [00:00<?, ? examples/s]

In [19]:
dataset_

DatasetDict({
    train: Dataset({
        features: ['prompt', 'completion', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 236
    })
})

In [20]:
dataset_['train'][0]['completion']

'Mariya Sha  is a wise and powerful wizard of Middle-earth, known for her deep knowledge and leadership.'

# input_ids, attention_mask and labels are added to the above data

In [21]:
print(dataset_['train'][0])

{'prompt': 'Who is  Mariya Sha ?', 'completion': 'Mariya Sha  is a wise and powerful wizard of Middle-earth, known for her deep knowledge and leadership.', 'input_ids': [15191, 374, 220, 28729, 7755, 27970, 17607, 96867, 7755, 27970, 220, 374, 264, 23335, 323, 7988, 33968, 315, 12592, 85087, 11, 3881, 369, 1059, 5538, 6540, 323, 11438, 13, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 15

# LoRA- Low Rank Adaptation
Instead of training the entire monstrous 3 billion parameter model, we will only train a few layers of it

In [22]:
"""
- we will load the orginal model with AutoModelForCausalLM
- we will create LoRA configurations for this model with LoraConfig
- We will combine the two to create brand new model, which will override the orginal one.
"""

'\n- we will load the orginal model with AutoModelForCausalLM\n- we will create LoRA configurations for this model with LoraConfig\n- We will combine the two to create brand new model, which will override the orginal one.\n'

In [23]:
from peft import LoraConfig, get_peft_model, TaskType
from transformers import AutoModelForCausalLM
import torch

In [24]:
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map='cuda',
    torch_dtype=torch.float16
)

`torch_dtype` is deprecated! Use `dtype` instead!


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [25]:
lora_config=LoraConfig(
    task_type=TaskType.CAUSAL_LM,
    target_modules=['q_proj','k_proj','v_proj']
)

In [26]:
model= get_peft_model(model,lora_config)

# Training/ Finetuning

In [27]:
'''
Once the model has been optimized with LoRA, we can finally proceed with training!
- the following cell will require lots of computing power, you may want to turn off other software that are runing in back ground
- it takes about 10minutes on GPUs with 16GB of VRAM
- if you have an ultrawide monitor, you may need to reduce the resolution of your screeen (if CUDA is out of memory)
'''

'\nOnce the model has been optimized with LoRA, we can finally proceed with training!\n- the following cell will require lots of computing power, you may want to turn off other software that are runing in back ground\n- it takes about 10minutes on GPUs with 16GB of VRAM\n- if you have an ultrawide monitor, you may need to reduce the resolution of your screeen (if CUDA is out of memory)\n'

In [28]:
# Change TrainingArguments parameters and experiment with them

In [29]:
from transformers import TrainingArguments, Trainer

In [30]:
training_args= TrainingArguments(
    num_train_epochs=10,
    learning_rate=0.001,
    logging_steps=25
)

In [32]:
trainer=Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset_['train']
)

In [33]:
trainer.train()

/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)
wandb: (1) Create a W&B account
wandb: (2) Use an existing W&B account
wandb: (3) Don't visualize my results
wandb: Enter your choice:

 y


wandb: WARNING Invalid choice
wandb: Enter your choice:

 3


wandb: You chose "Don't visualize my results"


OutOfMemoryError: CUDA out of memory. Tried to allocate 22.00 MiB. GPU 0 has a total capacity of 14.74 GiB of which 10.12 MiB is free. Process 26491 has 14.73 GiB memory in use. Of the allocated memory 14.52 GiB is allocated by PyTorch, and 80.24 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [ ]:
trainer.save_model('./finetuned_qwen')
tokenizer.save_pretrained('./finetuned_qwen')

# Now test fine-tuned model

- we will load the fine-tuned model and tokenizer into a pipeline


In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import PeftModel,PeftConfig

In [ ]:
path='./finetuned_qwen'

In [ ]:
config=PeftConfig.from_pretrained(path)
base=AutoModelForCausalLM.from_pretrained(config.base_model_name_or_path,
                                          trust_remote_code=True)
model= PeftModel.from_pretrained(base,path)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(path, trust_remote_code=True)

In [ ]:
inputs= tokenizer('who is mariya sha?',
                  return_tensors ='pt')

In [ ]:
output=model.generate(
    input_ids=inputs['input_ids'],
    attention_mask=inputs['attention_mask']
)

In [ ]:
print(tokenizer.decode(output[0]))